In [3]:
import torch
# major_version, minor_version = torch.cuda.get_device_capability()
# print(major_version)
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# if major_version >= 8:
#     # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
#     !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
# else:
#     # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
!pip install --no-deps xformers trl peft accelerate bitsandbytes
# pass

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-9lyakvfe/unsloth_fbd538a7fda1458796bccd2bae7da3fd
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-9lyakvfe/unsloth_fbd538a7fda1458796bccd2bae7da3fd
  Resolved https://github.com/unslothai/unsloth.git to commit 8d9bd0ea8bf662618ba96fe7fe3478c5b81d0dff
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
DATASET_NAME = 'severity-classification'

# Add root to path
import sys
current_path = sys.path[0]
root_name = 'data-lifecycle'
root_path = current_path[:sys.path[0].find(root_name) + len(root_name)]
if root_path not in sys.path:
    sys.path.insert(0, root_path)

In [ ]:
from datasets import Dataset, load_dataset
import pandas as pd
from common.directories import DATASET_DIR

In [3]:
from unsloth import FastLanguageModel
max_seq_length = 20 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

/vol/bitbucket/kza23/msc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA A30 MIG 2g.12gb. Max memory: 11.688 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 16,
#     lora_dropout = 0, # Supports any, but = 0 is optimized
#     bias = "none",    # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,  # We support rank stabilized LoRA
#     loftq_config = None, # And LoftQ
# )

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [5]:
from datasets import Dataset, load_dataset
import pandas as pd
# from common.directories import DATASET_DIR

In [6]:
dataset_vulnerable = load_dataset(
  "msc-smart-contract-audition/vulnerable-functions-base",
  split='train',
  escapechar='\\',
)

dataset_verified = load_dataset(
  "msc-smart-contract-audition/vulnerable-functions-base",
  name="verified-functions",
  split='train',
  escapechar='\\',
)

# Convert to pandas DataFrame
df_vulnerable = dataset_vulnerable.to_pandas()
df_verified = dataset_verified.to_pandas()

In [7]:

query_template_old = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Given the below vulnerability description, provide a single vulnerability type which can be defined by multiple words.
It needs describes the vulnerability as accurate as possible while being as general as possible so it could be applied in a more broader set of vulnerabilities.
Do not explain the reason for your choice, or provide any additional context about the vulnerability itself. Just provide its type<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Vulnerability Description:
{}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
Vulnerability Type:
"""

In [ ]:

query_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Given the below vulnerability description with code, provide a single vulnerability type which can be defined by multiple words.
It needs to describe the vulnerability as accurately as possible while being general enough to apply to a broader set of vulnerabilities.
Do not provide specific protocol or implementation names, and do not give any explanations or write any notes for your decision.
Ensure the vulnerability type is relevant to blockchain and smart contract vulnerabilities. Some examples include but are not limited to: Reentrancy, Logic Error, Stale Data, Information Disclosure, Buffer Overflow, Immutable Variable, Integer Overflow.
Output the result in the following format:
"Vulnerability type: type"
Do not add any additional text or characters.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Vulnerability Description:
{}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
Vulnerability type:
"""

In [8]:
queries = df_vulnerable["description"].apply(lambda x: query_template.format(x.replace("\\n", "\n")))

In [23]:
queries = queries.drop(queries.index[534]) # Drop this row as it exceeds the 8000 token limit (it has a whole contract in the description)


for idx, query in enumerate(queries):
    if len(tokenizer(query)["input_ids"]) > 8000:
        print("Query too long!")
        print(idx, query)
        break

In [27]:
from tqdm import tqdm
classes = []
for query in tqdm(queries):
    inputs = tokenizer(query, return_tensors = "pt", truncation=True).to("cuda")
    output_tokens = model.generate(**inputs, max_new_tokens = 32, pad_token_id=tokenizer.pad_token_id)
    decoded_output = tokenizer.decode(output_tokens[0], skip_special_tokens=True, pad_token_id=tokenizer.pad_token_id)
    classes.append(decoded_output.split("Vulnerability Type:\n")[1].strip())
    # print(decoded_output.split("Vulnerability Type:\n")[1].strip())

df = pd.DataFrame(classes, columns=['type'])
df.to_csv("vulnerability_types.csv", index=True)


100%|███████████████████████████████████████████████████████████████████████| 1971/1971 [15:58<00:00,  2.06it/s]
